<a href="https://colab.research.google.com/github/benza613/Sentiment-Analysis-using-Deep-Learning/blob/master/notebooks/LexicalAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Mount Google Drive
from google.colab import drive # import drive from google colab

ROOT = "/content/gdrive"     # default location for the drive
print(ROOT)                 # print content of ROOT (Optional)

drive.mount(ROOT)           # we mount the google drive at /content/drive

In [0]:
%cd gdrive/My\ Drive/SNLP

In [0]:
%cd Sentiment-Analysis-using-Deep-Learning/

In [0]:
import sys
import pickle
import random

#the preprocessed files are stored in Data_Balanced_20000_Apr-03-2020_23-22 folder 
#https://github.com/abdulfatir/twitter-sentiment-analysis/blob/master/code/preprocess.py

#the text files with positive and negative words in data_pos_neg folder
POSITIVE_WORDS_FILE = 'data_pos_neg/positive_words.txt'
NEGATIVE_WORDS_FILE = 'data_pos_neg/negative_words.txt'


Train_ZipCSVFileName = 'Train_100000_Apr-19-2020_22-33.csv'
Test_ZipCSVFileName = 'Test_2000_Apr-19-2020_22-40.csv'

import pandas as pd
import numpy as np


df_train = pd.read_csv(Train_ZipCSVFileName)
df_train.info()

df_test = pd.read_csv(Test_ZipCSVFileName)
df_test.info()

In [0]:
TRAIN = False

def save_results_to_csv(results, csv_file):
    ''' Save list of type [(reviewerID, prediction,overall_new)] to csv in Kaggle format '''
    with open(csv_file, 'w') as csv:
        csv.write('id,prediction,overall_new\n')
        for reviewerID, pred, overall in results:
            csv.write(reviewerID)
            csv.write(',')
            csv.write(str(pred))
            csv.write(',')
            csv.write(str(overall))
            csv.write('\n')

def file_to_wordset(filename):
    ''' Converts a file with a word per line to a Python set '''
    words = []
    with open(filename, 'r') as f:
        for line in f:
            words.append(line.strip())
    return set(words)

def classify(processed_csv, test_file=True, **params):
    positive_words = file_to_wordset(params.pop('positive_words'))
    print("positive")
    negative_words = file_to_wordset(params.pop('negative_words'))
    predictions = []
    overall_new = 0
    with open(processed_csv, 'r') as csv:
        for line in csv:
           # print("line",line)
            if test_file:
                overall, reviewTime1, reviewTime2, reviewerID,asin,reviewText,*rest = line.strip().split(',')
            else:
               # print(line.strip().split(','))
                overall, reviewTime1, reviewTime2, reviewerID, asin, reviewText,*rest = line.strip().split(',')
            pos_count, neg_count = 0, 0
            for word in reviewText.split():
                #print("word",word)
                if word in positive_words:
                    pos_count += 1
                elif word in negative_words:
                    neg_count += 1
            # print pos_count, neg_count
            #print("overall",type(overall)
                if overall == "5.0":
                    overall_new = 1

                if overall == "4.0":
                    overall_new = 1

                if overall == "3.0":
                    overall_new = 1

                if overall == "1.0":
                    overall_new = 0

                if overall == "2.0":
                    overall_new = 0

            prediction = 1 if pos_count >= neg_count else 0
           
            if test_file:
                predictions.append((reviewerID, prediction, overall_new))
            else:
                predictions.append((reviewerID, prediction, overall_new))
            #print("predictions",predictions)
    return predictions


if __name__ == '__main__':
    if TRAIN:
        predictions = classify(Train_ZipCSVFileName, test_file=(not TRAIN), positive_words=POSITIVE_WORDS_FILE, negative_words=NEGATIVE_WORDS_FILE)
        print(predictions)
        save_results_to_csv(predictions, 'train_baseline.csv')
        correct = sum([1 for p in predictions if p[1] == p[2]]) * 100.0 / len(predictions)
        print('Correct = %.2f%%' % correct)
    else:
        predictions = classify(Test_ZipCSVFileName, test_file=(not TRAIN), positive_words=POSITIVE_WORDS_FILE, negative_words=NEGATIVE_WORDS_FILE)
        correct = sum([1 for p in predictions if p[1] == p[2]]) * 100.0 / len(predictions)
        print('Correct = %.2f%%' % correct)